## АГРЕГАТНЫЕ ФУНКЦИИ

***Агрегатные функции*** помогают вычислять сводные значения для группы строк.

Мы будем работать с таблицей [sql.pokemon](http://sql.skillfactory.ru:3000/question#eyJkYXRhc2V0X3F1ZXJ5Ijp7ImRhdGFiYXNlIjoyLCJxdWVyeSI6eyJzb3VyY2UtdGFibGUiOjM3fSwidHlwZSI6InF1ZXJ5In0sImRpc3BsYXkiOiJ0YWJsZSIsInZpc3VhbGl6YXRpb25fc2V0dGluZ3MiOnt9fQ==), содержащей данные о покемонах и их характеристиках из классических видеоигр =^_-=

Если присмотреться к содержимому таблицы sql.pokemon: в ней хранится информация о покемонах.

Название поля	Содержимое

* id - уникальный идентификатор

* name - имя

* type1 - основной тип

* type2 - дополнительный тип

* hp - количество очков здоровья

* attack - показатели атаки

* defense - показатели защиты

* speed - показатели скорости

``` sql
-- Все основные типы покемонов

select 
type1,
type2
from sql.pokemon
```

Видно, что типы повторяются, потому что в результате запроса мы получаем все строки, которые подходят под заданные условия.

Чтобы получить уникальные значения из столбца, воспользуемся ключевым словом DISTINCT.

Примечание: DISTINCT пишется только один раз, в начале списка получаемых столбцов.

``` sql

-- Получим, например, все уникальные пары основного и дополнительного типов для покемонов.

select distinct
type1,
type2
from sql.pokemon
```

Слово DISTINCT применяется ко всем колонкам указынным после этого слова и ко всему выражению.

Кроме простых математических операций, СУБД позволяет проводить статистические вычисления для нескольких строк.

``` sql
-- Посчитаем количество строк в таблице. Для этого применим агрегатную функцию COUNT.

select count(*)
from sql.pokemon
```

Если в аргументе функции указать название столбца, функция обработает только строки с непустым значением.

Внутри функции COUNT мы можем также применять DISTINCT, чтобы вычислить количество уникальных значений.

``` sql
select 
count(distinct type2)
from sql.pokemon
```

## ОСНОВНЫЕ АГРЕГАТНЫЕ ФУНКЦИИ

Основные агрегатные функции, с которыми предстоит работать:

* COUNT — вычисляет число непустых строк;

* SUM — вычисляет сумму;

* AVG — вычисляет среднее;

* MAX — вычисляет максимум;

* MIN — вычисляет минимум.

``` sql
-- Cумма здоровья всех покимонов в таблице

select 
sum (hp)
from sql.pokemon

-- Среднее значение здоровья всех покимонов в таблице

select 
avg (hp)
from sql.pokemon

-- Максимум значения здоровья всех покимонов в таблице

select 
max (hp)
from sql.pokemon

-- Минимум значения здоровья всех покимонов в таблице

select 
min (hp)
from sql.pokemon

-- Среднее количество очков здоровья у покемонов-драконов (то есть тех, у кого основной тип — Dragon)

select
avg (hp)
from sql.pokemon
where type1 = 'Dragon'
```

Кроме того, можно применять несколько агрегатных функций в одном запросе.

Посчитаем количество трявяных покемонов (тип - Grass), сколько среди них с дополнительным типом (type2 не NUll), средняя атака, средняя защита

``` sql
select
count (*) as "Всего травяных покемонов",
count (type2) as "Покемонов с дополнительным типом",
avg (attack) as "Средняя атака",
avg (defense) as "Cредняя защита"
from sql.pokemon
where type1 = 'Grass'

-- Запрос, который выведет:
-- количество покемонов (столбец pokemon_count),
-- среднюю скорость (столбец avg_speed),
-- максимальное и минимальное число очков здоровья (столбцы max_hp и min_hp)
-- для электрических (Electric) покемонов, имеющих дополнительный тип и показатели атаки или защиты больше 50.

select
count(*) as "pokemon_count",
avg(speed) as "avg_speed",
max(hp) as "max_hp",
min(hp) as "min_hp"
from sql.pokemon
where 
type1 = 'Electric'
and type2 is not null
and (attack > 50 or defense > 50)
```

## ГРУППИРОВКА
GROUP BY используется для определения групп выходных строк, к которым могут применяться агрегатные функции.

GROUP BY пишется в самом конце и включает все колонки, которые не участвуют в агрегации.

Уровень агрегации должен быть одинаковым.

``` sql
-- GROUP BY. Какое количество типов покимонов в таблице каждого типа

select
type1 as "Осн. тип",
type2 as "Доп. тип",
count (*) as "Количество типов"
from sql.pokemon
group by type1, type2
order by type1, type2 nulls first

-- GROUP BY без агрегатной функции (вывод без повторов). Запрос аналогичес с DISTINCT

select
type1
from sql.pokemon
group by type1

-- Запрос, который выведет:
-- число различных дополнительных типов (столбец additional_types_count),
-- среднее число очков здоровья (столбец avg_hp),
-- сумму показателей атаки (столбец attack_sum) в разбивке по основным типам (столбец primary_type).
-- Отсортируйте результат по числу дополнительных типов в порядке убывания, при равенстве — по основному типу в алфавитном порядке.
-- Столбцы к выводу (обратите внимание на порядок!): primary_type, additional_types_count, avg_hp, attack_sum.

select
type1 as primary_type,
count(distinct type2) as additional_types_count,
avg(hp) as avg_hp,
sum(attack) as attack_sum
from sql.pokemon
group by primary_type
order by additional_types_count desc,
primary_type asc;


-- Мы можем осуществлять группировку по нескольким столбцам.

select
type1 as primary_type,
type2 as additional_type
from sql.pokemon
group by primary_type, additional_type
order by primary_type, additional_type nulls first
```

## ФИЛЬТРАЦИЯ АГРЕГИРОВАННЫХ СТРОК

Если ключевое слово WHERE определяет фильтрацию строк до агрегирования, то для фильтрации уже агрегированных данных применяется ключевое слово HAVING.

HAVING пишется после GROUP BY, но не после ORDER BY (если присутствует)!

``` sql

-- Отсеиваем количество типов больше 50

select
type1,
count(*) as count_type
from sql.pokemon
group by type1
having count(*) > 50

-- Cредняя атака определенного типа для тех случаев когда средняя атака больше 90

select
type1 as primary_type,
avg(attack)
from sql.pokemon
group by type1
having avg(attack) > 90
order by avg(attack)
```

В HAVING вы можете использовать все те же условия, что и в WHERE.

В общем виде синтаксис оператора SELECT, с учётом имеющихся на данный момент знаний, представляется следующим образом:

``` sql
SELECT [ALL | DISTINCT] список_столбцов|*
FROM список_имён_таблиц
[WHERE условие_поиска]
[GROUP BY список_имён_столбцов]
[HAVING условие_поиска]
[ORDER BY имя_столбца [ASC | DESC],…]
```

Примечание: В квадратных скобках указаны необязательные предложения: они могут отсутствовать в операторе SELECT

``` sql

-- Запрос, который выведет основной и дополнительный типы покемонов (столбцы primary_type и additional_type) 
-- для тех, у кого средний показатель атаки больше 100 и максимальный показатель очков здоровья меньше 80.

select
type1 as primary_type,
type2 as additional_type
from sql.pokemon
group by primary_type, additional_type
having avg(attack) > 100 
and max(hp) < 80

-- Запрос, чтобы для покемонов, чьё имя (name) начинается с S, вывести столбцы с их основным типом (primary_type) и 
-- общим числом покемонов этого типа (pokemon_count). 
-- Оставьте только те типы, у которых средний показатель защиты больше 80. 
-- Выведите топ-3 типов по числу покемонов в них.

select
type1 as primary_type,
count(*) as pokemon_count
from sql.pokemon
where name like 'S%'
group by type1
having avg(defense) > 80
order by pokemon_count desc
limit 3

-- Сколько РАЗЛИЧНЫХ (distinct) значений показателей атаки есть у покемонов с типом Water (основным или дополнительным)?

select
count(distinct attack) unique_attack_count
from sql.pokemon
where type1 = 'Water' or type2 = 'Water'

-- Запрос, который выведет основной и дополнительный типы покемонов и средние значения по каждому показателю 
-- (столбцы avg_hp, avg_attack, avg_defense, avg_speed). 
-- Оставьте только те пары типов, у которых сумма этих четырёх показателей более 400.

select 
type1 as primary_type,
type2 as additional_type,
avg(hp) as avg_hp,
avg(attack) as avg_attack,
avg(defense) as avg_defense,
avg(speed) as avg_speed
from sql.pokemon
group by primary_type, additional_type
having avg(hp) + avg(attack) + avg(defense) + avg(speed) > 400

-- Запрос, который выведет столбцы с основным типом покемона и общим количеством покемонов этого типа. 
-- Учитывайте только тех покемонов, у кого или показатель атаки, или показатель защиты принимает значение между 50 и 100 включительно. 
-- Оставьте только те типы покемонов, у которых максимальный показатель здоровья не больше 125. 
-- Выведите только тот тип, который находится на пятом месте по количеству покемонов.

select 
type1 as primary_type,
count(*) as cnt
from sql.pokemon
where attack between 50 and 100 or defense between 50 and 100
group by primary_type
having max(hp) <= 125
order by count(*) desc
limit 1 offset 4

```